In [1]:
import pandas as pd
import numpy as np

df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')

C:\Users\alova\AppData\Local\Temp\ipykernel_10856\2594960417.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')


In [2]:
#Seleccionamos las columnas a usar
df_people_driver_info = df_people[['PERSON_ID', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT', 'CELL_PHONE_USE', 'DRIVERS_LICENSE_CLASS']]
df_people_driver_info.shape

(2190983, 7)

In [3]:
for c in df_people_driver_info:
    print("------- Atributo:", c)
    print("Valores nulos: ", df_people_driver_info[c].isna().sum())
    print("Valores distintos: ", df_people_driver_info[c].unique())


------- Atributo: PERSON_ID
Valores nulos:  0
Valores distintos:  ['O2179723' 'P480828' 'P480829' ... 'O595730' 'O595731' 'P134373']
------- Atributo: DRIVER_ACTION
Valores nulos:  446163
Valores distintos:  ['UNKNOWN' nan 'OTHER' 'NONE' 'OVERCORRECTED' 'FOLLOWED TOO CLOSELY'
 'FAILED TO YIELD' 'IMPROPER BACKING' 'IMPROPER TURN' 'IMPROPER PASSING'
 'DISREGARDED CONTROL DEVICES' 'EVADING POLICE VEHICLE'
 'IMPROPER LANE CHANGE' 'TOO FAST FOR CONDITIONS'
 'CELL PHONE USE OTHER THAN TEXTING' 'EMERGENCY VEHICLE ON CALL'
 'WRONG WAY/SIDE' 'TEXTING' 'IMPROPER PARKING' 'STOPPED SCHOOL BUS'
 'LICENSE RESTRICTIONS']
------- Atributo: DRIVER_VISION
Valores nulos:  446876
Valores distintos:  ['NOT OBSCURED' nan 'UNKNOWN' 'PARKED VEHICLES' 'MOVING VEHICLES'
 'BLINDED - SUNLIGHT' 'OTHER' 'SIGNBOARD' 'WINDSHIELD (WATER/ICE)'
 'BLINDED - HEADLIGHTS' 'TREES, PLANTS' 'BUILDINGS' 'EMBANKMENT'
 'HILLCREST' 'BLOWING MATERIALS']
------- Atributo: PHYSICAL_CONDITION
Valores nulos:  444978
Valores distintos: 

In [4]:
df_people_driver_info.columns = (
    df_people_driver_info.columns
      .astype(str)
      .str.replace("\ufeff", "", regex=False)       
      .str.replace(r"^[^A-Za-z0-9]+", "", regex=True) 
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)         
)

In [5]:
#convertir los datos basura a None, para que en postgres se consideren NULL
df_people_driver_info.replace(['', ' ', 'NA', 'N/A', 'null', 'NULL'], None)
df_people_driver_info = df_people_driver_info.where(df_people_driver_info.notna(), None) 

In [6]:
print(df_people_driver_info.columns)

Index(['PERSON_ID', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION',
       'BAC_RESULT', 'CELL_PHONE_USE', 'DRIVERS_LICENSE_CLASS'],
      dtype='object')


In [7]:
#Cambiamos los tipos de datos 
df_people_driver_info = df_people_driver_info.assign(
    PERSON_ID = df_people_driver_info['PERSON_ID'].astype('string'),
    DRIVER_ACTION = df_people_driver_info['DRIVER_ACTION'].astype('string'),
    DRIVER_VISION = df_people_driver_info['DRIVER_VISION'].astype('string'),
    PHYSICAL_CONDITION = df_people_driver_info['PHYSICAL_CONDITION'].astype('string'),
    BAC_RESULT = df_people_driver_info['BAC_RESULT'].astype('string'),
    CELL_PHONE_USE = df_people_driver_info['CELL_PHONE_USE'].astype('string'),
    DRIVERS_LICENSE_CLASS = df_people_driver_info['DRIVERS_LICENSE_CLASS'].astype('string')
)

In [8]:
import re

# limpiamos los valores inválidos del atributo DRIVERS_LICENCE_CLASS  
df_people_driver_info['DRIVERS_LICENSE_CLASS'] = (
    df_people_driver_info['DRIVERS_LICENSE_CLASS']
    .astype("string")
    .str.upper()
    .str.strip()
    .apply(lambda x: x if pd.notna(x) and re.fullmatch(r'[A-Z0-9]+', x) else None)
)

In [9]:
#verificamos
df_people_driver_info['DRIVERS_LICENSE_CLASS'].unique()

array(['D', None, 'A', 'DA', 'EO', 'DM', 'C', 'AM', 'B', 'E', 'M', 'BM',
       'O', 'DV', 'DL', '0', 'UN', 'CD', 'N', 'CM', 'TM', 'U', 'IL', 'UK',
       'R', 'NO', 'CA', 'DD', 'F', '1', 'AB', 'AR', 'AL', 'K', 'LP', 'H',
       'G', '9', 'WA', 'D7', 'PS', 'L', 'C2', 'J', 'S', 'P', '99', 'MD',
       'C1', 'HN', 'MC', 'I', 'DC', '10', 'A3', 'D9', 'CH', 'DE', 'XD',
       'M2', 'CL', '1F', 'TV', 'MP', '5', 'X', 'BF', 'BC', 'IP', 'DN',
       'LN', 'BW', 'V', '3', 'EC', 'DR', 'KM', 'TL', 'NT', 'ND', 'BE',
       'AC', '00', 'G1', 'TD', 'DQ', 'OA', 'P2', 'D5', 'NP', 'Y', 'RG',
       'CP', 'T', 'DS', 'RE', 'UB', 'NC', '2', 'ID', 'SB', 'CB', 'NV',
       'AA', 'CO', 'NM', 'W', 'BP', 'TP', 'Q', 'P1', '4', 'PR', 'AP',
       'ST', 'DF', 'VL', '6', 'BL', 'GM', 'LT', '25', 'DJ', 'A0', 'DB',
       'PA', 'Z', 'XX', 'RF', 'B1', 'PV', 'EM', 'M1', '4D', 'DH', '1B',
       'DU', '01', 'E0', 'FD', 'JR', 'DZ', 'DW', '05', 'BG', '2D', 'AD',
       'A1', 'MO', 'SC', 'LM', 'CQ', '56', '31', 'EX', 'SD', 

In [ ]:
#reemplazamos los valores de unknown en DRIVER ACTION 
df_people_driver_info['DRIVER_ACTION'] = df_people_driver_info['DRIVER_ACTION'].replace("UNKNOWN", None)
df_people_driver_info['DRIVER_ACTION'].unique()

<StringArray>
[                               <NA>,                             'OTHER',
                              'NONE',                     'OVERCORRECTED',
              'FOLLOWED TOO CLOSELY',                   'FAILED TO YIELD',
                  'IMPROPER BACKING',                     'IMPROPER TURN',
                  'IMPROPER PASSING',       'DISREGARDED CONTROL DEVICES',
            'EVADING POLICE VEHICLE',              'IMPROPER LANE CHANGE',
           'TOO FAST FOR CONDITIONS', 'CELL PHONE USE OTHER THAN TEXTING',
         'EMERGENCY VEHICLE ON CALL',                    'WRONG WAY/SIDE',
                           'TEXTING',                  'IMPROPER PARKING',
                'STOPPED SCHOOL BUS',              'LICENSE RESTRICTIONS']
Length: 20, dtype: string

In [ ]:
#reemplazamos los valores de unknown en el BAC result
df_people_driver_info['DRIVER_VISION'] = df_people_driver_info['DRIVER_VISION'].replace("UNKNOWN", None)
df_people_driver_info['DRIVER_VISION'].unique()

In [10]:
df_people_driver_info.to_csv("people_driver_info.csv")
